### Packages

In [2]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from fiona.crs import from_epsg
from tqdm import tqdm
from shapely.geometry import Point

# Current working directory
basepath = Path.cwd().parent.parent.parent

# Read list of stations within model coverage area
raw_data = os.path.join(basepath, '01 Raw Data' )

### Paths

### Read CSMT Zone shapefiles

In [3]:
### Load CSMT Zoning Structure
# Load zone  shapefile
csmt_zones = gpd.read_file(f'{raw_data}/16 CSMT/Shapefiles/zonecentroid_08032024_w demand3_zone.SHP')
crs = csmt_zones.crs
print(crs)
csmt_zones.rename(columns={'NO':'zone_no','MODEL_AREA':'model_area','XCOORD':'x_coord','YCOORD':'y_coord'},inplace=True)
csmt_zones = csmt_zones[['zone_no','model_area','x_coord','y_coord','geometry']]
csmt_zones.head()

PROJCS["British_National_Grid_TOWGS",GEOGCS["OSGB36",DATUM["Ordnance_Survey_of_Great_Britain_1936",SPHEROID["Airy 1830",6377563.396,299.3249646,AUTHORITY["EPSG","7001"]],TOWGS84[446.4,-125.2,542.1,0.15,0.247,0.842,-20.49],AUTHORITY["EPSG","6277"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",49],PARAMETER["central_meridian",-2],PARAMETER["scale_factor",0.999601272],PARAMETER["false_easting",400000],PARAMETER["false_northing",-100000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


,zone_no,model_area,x_coord,y_coord,geometry
0,101,External,483442.337,358139.545,"MULTIPOLYGON (((401377.701 368767.099, 401375...."
1,201,External,571805.254,263684.340,"MULTIPOLYGON (((552628.541 325295.972, 552876...."
2,301,External,548177.927,143853.334,"MULTIPOLYGON (((488482.073 151931.887, 488462...."
3,401,External,434860.371,494161.164,"POLYGON ((531775.041 407720.939, 533582.045 40..."
4,505,External,352895.029,461898.387,"MULTIPOLYGON (((331957.783 391954.992, 331916...."


### Mapping LSOA to CSMT
- Can be avoided as it is a one time process

In [4]:
### Read LSOA shapefile
lsoa_shp = gpd.read_file(f'{raw_data}/08 Pop data/LSOA_2021_EW_BGC.shp').to_crs(crs=crs)
print(lsoa_shp.crs)
lsoa_shp = lsoa_shp[['LSOA21CD','LSOA21NM','geometry']]
lsoa_shp.head()

PROJCS["British_National_Grid_TOWGS",GEOGCS["OSGB36",DATUM["Ordnance_Survey_of_Great_Britain_1936",SPHEROID["Airy 1830",6377563.396,299.3249646,AUTHORITY["EPSG","7001"]],TOWGS84[446.4,-125.2,542.1,0.15,0.247,0.842,-20.49],AUTHORITY["EPSG","6277"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",49],PARAMETER["central_meridian",-2],PARAMETER["scale_factor",0.999601272],PARAMETER["false_easting",400000],PARAMETER["false_northing",-100000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


,LSOA21CD,LSOA21NM,geometry
0,E01000001,City of London 001A,"POLYGON ((532105.312 182010.574, 532162.491 18..."
1,E01000002,City of London 001B,"POLYGON ((532634.497 181926.016, 532619.141 18..."
2,E01000003,City of London 001C,"POLYGON ((532135.138 182198.131, 532158.250 18..."
3,E01000005,City of London 001E,"POLYGON ((533808.018 180767.774, 533649.037 18..."
4,E01000006,Barking and Dagenham 016A,"POLYGON ((545122.049 184314.931, 545271.849 18..."


In [5]:
###Mapping using Postcode as proxy
open_pc_gdf = gpd.read_file(f'{basepath}/01 Raw Data/08 Pop data/Post Code data/codepo_gpkg_gb/Data/codepo_gb.gpkg')
open_pc_gdf = open_pc_gdf[['postcode','geometry']].to_crs(crs=crs)
# print(open_pc_gdf.crs)
open_pc_gdf.head()

KeyboardInterrupt: 

In [6]:
# Initialize an empty GeoDataFrame to store the intersections
intersection_gdf = gpd.GeoDataFrame()
csmt_zones['geometry'] = csmt_zones['geometry'].buffer(0)
lsoa_shp['geometry'] = lsoa_shp['geometry'].buffer(0)
# Iterate over each pair of geometries and calculate the intersections
for idx1, geometry1 in tqdm(csmt_zones.iterrows(), total = len(csmt_zones)):
    for idx2, geometry2 in lsoa_shp.iterrows():
        intersection = geometry1['geometry'].intersection(geometry2['geometry'])
        
        # Check if the intersection is not empty
        if not intersection.is_empty:
            intersection_gdf = pd.concat([intersection_gdf,(gpd.GeoDataFrame({'zone': geometry1['zone_no'],
                                                                              'model_area': geometry1['model_area'],
                                                                              'LSOA21CD':geometry2['LSOA21CD'],
                                                                              'geometry': [intersection]},crs=crs))])

intersection_gdf.drop_duplicates(inplace=True)
intersection_gdf.head()

 51%|█████     | 429/846 [24:24<23:43,  3.41s/it] 


KeyboardInterrupt: 

In [9]:
points_within_polygons = gpd.sjoin( open_pc_gdf, intersection_gdf, how='inner', predicate='within')
grouped_pc =  points_within_polygons.groupby(['zone','LSOA21CD']).agg(pc_count = ('postcode','count')).reset_index()
grouped_pc.head()


,zone,LSOA21CD,pc_count
0,101,E01005908,3
1,101,E01005912,5
2,101,E01007610,1
3,101,E01007835,1
4,101,E01007842,1


In [10]:
merged_gdf = intersection_gdf.merge(grouped_pc, how='left', left_on=['zone', 'LSOA21CD'], right_on=['zone', 'LSOA21CD'])
merged_gdf['pc_count'].fillna(0, inplace=True)
merged_gdf.head()

,zone,model_area,LSOA21CD,geometry,pc_count
0,101,External,E01005409,"MULTIPOLYGON (((403042.041 400862.942, 403572....",0.0
1,101,External,E01005880,"MULTIPOLYGON (((399429.039 391090.939, 399398....",0.0
2,101,External,E01005908,"MULTIPOLYGON (((398038.575 386067.967, 398037....",3.0
3,101,External,E01005912,"POLYGON ((397549.074 386116.903, 397518.543 38...",5.0
4,101,External,E01006045,"POLYGON ((398381.075 392580.902, 398381.110 39...",0.0


In [11]:
merged_gdf_fil = merged_gdf[merged_gdf['pc_count']!= 0].copy()
lsoa_pc_count = merged_gdf.groupby('LSOA21CD').agg(lsoa_count = ('pc_count','sum')).reset_index()
lsoa_pc_count.head()

,LSOA21CD,lsoa_count
0,E01000001,55.0
1,E01000002,80.0
2,E01000003,35.0
3,E01000005,95.0
4,E01000006,14.0


In [12]:
zone_lsoa_pc = pd.merge(merged_gdf_fil, lsoa_pc_count, how='left', on='LSOA21CD')
zone_lsoa_pc['lsoa_count'].fillna(0, inplace=True)
zone_lsoa_pc['overlap'] = zone_lsoa_pc['pc_count']/zone_lsoa_pc['lsoa_count']
zone_lsoa_pc.head()

,zone,model_area,LSOA21CD,geometry,pc_count,lsoa_count,overlap
0,101,External,E01005908,"MULTIPOLYGON (((398038.575 386067.967, 398037....",3.0,50.0,0.060000
1,101,External,E01005912,"POLYGON ((397549.074 386116.903, 397518.543 38...",5.0,53.0,0.094340
2,101,External,E01007610,"MULTIPOLYGON (((463845.046 392969.941, 463848....",1.0,73.0,0.013699
3,101,External,E01007835,"POLYGON ((437663.086 382823.391, 437724.015 38...",1.0,19.0,0.052632
4,101,External,E01007842,"POLYGON ((439792.010 382719.905, 440308.077 38...",1.0,27.0,0.037037


In [17]:
zone_lsoa_pc.to_file(f'{basepath}/03 Output/09 Swift in CSMT/csmt_lsoa.shp',encoding='utf-8')
zone_lsoa_pc[['zone','model_area','LSOA21CD','pc_count','lsoa_count','overlap']].to_csv(f'{basepath}/03 Output/09 Swift in CSMT/csmt_lsoa.csv')

### Directly read the mapping of LSOA to CSMT
- Avoid previous time/memory intensive steps above 

In [7]:
zone_lsoa_pc = gpd.read_file(f'{basepath}/03 Output/09 Swift in CSMT/csmt_lsoa.shp').to_crs(crs=crs)
zone_lsoa_pc.head()

,zone,model_area,LSOA21CD,pc_count,lsoa_count,overlap,geometry
0,101,External,E01005908,3.0,50.0,0.060000,"MULTIPOLYGON (((398038.575 386067.967, 398037...."
1,101,External,E01005912,5.0,53.0,0.094340,"POLYGON ((397549.074 386116.903, 397518.543 38..."
2,101,External,E01007610,1.0,73.0,0.013699,"MULTIPOLYGON (((463845.046 392969.941, 463848...."
3,101,External,E01007835,1.0,19.0,0.052632,"POLYGON ((437663.086 382823.391, 437724.015 38..."
4,101,External,E01007842,1.0,27.0,0.037037,"POLYGON ((439792.010 382719.905, 440308.077 38..."


### Read OA data for mapping employment data and student activity

In [8]:
# Load zone  shapefile
oa_shp = gpd.read_file(f'{raw_data}/19 Employment activity/Output_Areas_2021_EW_BGC_V2/OA_2021_EW_BGC_V2.shp')
oa_shp.head()

,OA21CD,LSOA21CD,LSOA21NM,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,GlobalID,geometry
0,E00000001,E01000001,City of London 001A,None,532250,181864,51.5202,-0.09523,3a44dd3d-5082-4a09-9b9c-3a5fadc811ed,"POLYGON ((532303.492 181814.110, 532213.378 18..."
1,E00000003,E01000001,City of London 001A,None,532171,181819,51.5198,-0.09638,f1216dc8-14d1-4857-9230-cab0641758fb,"POLYGON ((532213.378 181846.192, 532190.539 18..."
2,E00000005,E01000001,City of London 001A,None,532166,181722,51.5190,-0.09649,44d6f70f-549c-4288-9b6d-de2adbf02582,"POLYGON ((532180.131 181763.020, 532219.161 18..."
3,E00000007,E01000001,City of London 001A,None,532088,181473,51.5167,-0.09771,4dd683e1-9a5c-46cf-9e19-8465c8fbb6cb,"POLYGON ((532201.292 181668.180, 532267.728 18..."
4,E00000010,E01000003,City of London 001C,None,532092,182114,51.5225,-0.09741,7476781f-8fe4-4c9b-bde1-0eecbd146dff,"POLYGON ((532127.958 182133.192, 532089.264 18..."


In [9]:
oa_shp = oa_shp[['OA21CD','geometry']].copy()
oa_shp['oa_area'] = oa_shp.geometry.area
oa_shp.head()

,OA21CD,geometry,oa_area
0,E00000001,"POLYGON ((532303.492 181814.110, 532213.378 18...",6949.151483
1,E00000003,"POLYGON ((532213.378 181846.192, 532190.539 18...",4492.411068
2,E00000005,"POLYGON ((532180.131 181763.020, 532219.161 18...",8565.514208
3,E00000007,"POLYGON ((532201.292 181668.180, 532267.728 18...",75994.829702
4,E00000010,"POLYGON ((532127.958 182133.192, 532089.264 18...",2102.876614


In [12]:
### Read population data 
oa_emp = pd.read_csv(f'{raw_data}/19 Employment activity/1072552118067204.csv',
                       skiprows=6, 
                       skipfooter=7, 
                       engine='python')
oa_emp.head()                    
# oa_emp.rename(columns={'LSOA Code':'LSOA21CD'},inplace=True)
# lsoa_pop.head()

,2021 output area,mnemonic,Economically active and a full-time student,Economically inactive: Student
0,E00000001,E00000001,0,5
1,E00000003,E00000003,1,19
2,E00000005,E00000005,1,3
3,E00000007,E00000007,0,11
4,E00000010,E00000010,1,8


In [ ]:
oa_shp['geometry'] = lsoa_shp['geometry'].buffer(0)

# Initialize an empty GeoDataFrame to store the intersections
intersection_oa_gdf = gpd.GeoDataFrame()

# Iterate over each pair of geometries and calculate the intersections
for idx1, geometry1 in tqdm(zone_lsoa_pc.iterrows(), total = len(zone_lsoa_pc)):
    for idx2, geometry2 in oa_shp.iterrows():
        intersection2 = geometry1['geometry'].intersection(geometry2['geometry'])
        
        # Check if the intersection is not empty
        if not intersection2.is_empty:
            intersection_oa_gdf = pd.concat([intersection_oa_gdf,(gpd.GeoDataFrame({'zone': geometry1['zone_no'],
                                                                              'model_area': geometry1['model_area'],
                                                                              'LSOA21CD':geometry1['LSOA21CD'],
                                                                              'OA21CD': geometry2['OA21CD'],
                                                                              'geometry': [intersection]},crs=crs))])

intersection_oa_gdf.drop_duplicates(inplace=True)
intersection_oa_gdf.head()

In [ ]:
### Calculate size of overlap intersection polygons


intersection_oa_gdf['int_area'] = intersection_oa_gdf.geometry.area

merged_oa_gdf = pd.merge(intersection_oa_gdf, oa_shp[['OA21CD','oa_area']], how='inner', on='OA21CD')
merged_oa_gdf['overlap_emp'] = (merged_oa_gdf['int_area'] / merged_oa_gdf['oa_area']) * 100

### Remove redundant attributes
disagg_gdf = merged_gdf[['zone','model_area','LSOA21CD','int_area','overlap_per']].copy()
disagg_gdf.drop_duplicates(inplace=True)
disagg_gdf.head()